In [219]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [220]:
data = pd.read_csv('data1.csv', header = None)
label = pd.read_csv('label1.csv', header = None)

In [221]:
print(data.shape, label.shape)

(20000, 20) (20000, 1)


In [222]:
print(pd.DataFrame(data))

             0         1         2         3         4         5         6   \
0     -0.446891 -0.013397  0.232645  2.156649  1.652923 -0.210531 -0.662227   
1      0.243747 -1.144175 -0.622214 -0.661979 -0.373315  0.520313 -1.307954   
2     -1.417799 -0.088833 -0.647181 -1.141497 -1.321100  0.686798 -1.045569   
3      0.656784  0.381201 -1.039011  1.285315  0.808324  1.428519 -1.144367   
4      1.098520  1.706769 -1.030370  2.001009  2.751551  1.202229 -0.941066   
...         ...       ...       ...       ...       ...       ...       ...   
19995  0.146324 -0.618555  0.547413  0.851301  0.838880 -1.248402  0.698199   
19996 -2.227923 -0.327196  1.258849 -0.945254  3.925262  0.316686 -1.061159   
19997 -0.800405  1.477997  0.090057 -0.973997 -0.204970  0.224539  0.302265   
19998 -1.217344 -0.785401 -0.413781 -0.335324  0.181671 -1.685733 -0.437981   
19999  1.601005 -0.890398 -1.425049  0.887501  0.167205 -0.395042 -0.512554   

             7         8         9         10      

In [223]:
data = data.values
label = label.values

In [224]:
print(np.unique(label), np.max(label))

[0 1 2 3 4] 4


In [225]:
one_hot_label = np.eye(np.max(label) + 1)[label]

In [226]:
one_hot_label

array([[[0., 0., 0., 1., 0.]],

       [[0., 0., 1., 0., 0.]],

       [[0., 0., 1., 0., 0.]],

       ...,

       [[0., 0., 1., 0., 0.]],

       [[0., 0., 1., 0., 0.]],

       [[1., 0., 0., 0., 0.]]])

In [227]:
one_hot_label.shape

(20000, 1, 5)

In [228]:
one_hot_label = one_hot_label.reshape(20000, 5)

In [229]:
one_hot_label.shape

(20000, 5)

In [230]:
one_hot_label[0].shape

(5,)

In [231]:
train_data, train_data_dash = train_test_split(data, train_size = 0.8, shuffle = False)
train_label, train_label_dash = train_test_split(one_hot_label, train_size = 0.8, shuffle = False)

In [232]:
valid_data, test_data = train_test_split(train_data_dash, train_size = 0.5, shuffle = False)
valid_label, test_label = train_test_split(train_label_dash, train_size = 0.5, shuffle = False)

In [233]:
print(train_data.shape, train_label.shape, test_data.shape, test_label.shape, valid_data.shape, valid_label.shape)

(16000, 20) (16000, 5) (2000, 20) (2000, 5) (2000, 20) (2000, 5)


In [234]:
hidden_neurons = 10

In [235]:
w1 = np.random.normal(0, 1, (20, hidden_neurons))
w2 = np.random.normal(-1, 1, (hidden_neurons, 5))

In [236]:
b1 = np.zeros(hidden_neurons)
b2 = np.zeros(5)

In [237]:
def activation_function(data, choice):
    if choice == 0:
       return np.tanh(data)
    else:
      numerator = np.exp(data)
      return numerator/np.sum(numerator, axis = 1, keepdims=True)

In [238]:
lr_rate = 0.003
batch = 1000
epochs = 100

In [239]:
def cost_func(label, pred):
    return -(label * np.log(pred))

In [240]:
def forward_prop(data):
    a0 = data
    z1 = np.dot(a0, w1) + b1
    a1 = activation_function(z1, 0)
    z2 = np.dot(a1, w2) + b2
    a2 = activation_function(z2, 1)
    return a0, z1, a1, z2, a2

In [241]:
def accuracy(label, pred, size):
    pred_label = np.argmax(pred, axis = 1)
    actual_label = np.argmax(label, axis = 1)
    compare = np.equal(pred_label, actual_label)
    count = np.sum(compare)
    return count/size

In [242]:
for epoch in range(epochs):
    for iteration in range(int(train_data.shape[0]/batch)):
        a0, z1, a1, z2, a2 = forward_prop(train_data[iteration * batch : (iteration + 1) * batch, :])
        pred = a2
        grount_truth = train_label[iteration * batch : (iteration + 1) * batch, :]
        del2 = (pred - grount_truth)
        del1 = np.dot(del2, w2.T) * (1 - a1**2)
        dJdw2 = np.dot(a1.T, del2)
        dJdw1 = np.dot(a0.T, del1)
        dJdb2 = np.sum(del2, axis = 0)
        dJdb1 = np.sum(del1, axis = 0)
        w2 = w2 - lr_rate * dJdw2
        w1 = w1 - lr_rate * dJdw1
        b2 = b2 - lr_rate * dJdb2
        b1 = b1 - lr_rate * dJdb1

    a0, z1, a1, z2, a2 = forward_prop(train_data)
    print(f'Cost after epoch: {epoch}: {np.sum(cost_func(train_label, a2))/16000}')
    print(f'Accuracy after epoch: {epoch}: {accuracy(train_label, a2, 16000)}')

    a0, z1, a1, z2, a2 = forward_prop(valid_data)
    print(f'VALIDATION Cost after epoch: {epoch}: {np.sum(cost_func(valid_label, a2))/2000}')
    print(f'VALIDATION Accuracy after epoch: {epoch}: {accuracy(valid_label, a2, 2000)}')

Cost after epoch: 0: 0.825975687200273
Accuracy after epoch: 0: 0.691875
VALIDATION Cost after epoch: 0: 0.8588268972392987
VALIDATION Accuracy after epoch: 0: 0.683
Cost after epoch: 1: 0.6985589248048046
Accuracy after epoch: 1: 0.744875
VALIDATION Cost after epoch: 1: 0.7347671599712535
VALIDATION Accuracy after epoch: 1: 0.738
Cost after epoch: 2: 0.6073691483169297
Accuracy after epoch: 2: 0.7806875
VALIDATION Cost after epoch: 2: 0.6503422290838373
VALIDATION Accuracy after epoch: 2: 0.769
Cost after epoch: 3: 0.5461462984539435
Accuracy after epoch: 3: 0.803125
VALIDATION Cost after epoch: 3: 0.5883630109310427
VALIDATION Accuracy after epoch: 3: 0.791
Cost after epoch: 4: 0.5146790101535567
Accuracy after epoch: 4: 0.816125
VALIDATION Cost after epoch: 4: 0.552436343742208
VALIDATION Accuracy after epoch: 4: 0.81
Cost after epoch: 5: 0.4942711826178539
Accuracy after epoch: 5: 0.82425
VALIDATION Cost after epoch: 5: 0.5257986073530521
VALIDATION Accuracy after epoch: 5: 0.8235
